In [1]:
import csv
import StringIO
import json
from pyspark import SparkContext
from pyspark.sql import SQLContext
inputFile = './Prt5.txt'
#sc = SparkContext()
sc

ImportError: No module named pyspark

In [ ]:
%cat Prt5.txt

In [ ]:
def loadRecord(line):
    ''' Parse a CSV line'''
    #print line
    input = StringIO.StringIO(line)
    reader  = csv.DictReader(input, fieldnames= fields, delimiter = '\t')
    return reader.next()

raw_input = sc.textFile(inputFile)
inputHeader = raw_input.first()
fields = [el for el in inputHeader.split('\t')]
inputNoheader = raw_input.subtract( sc.parallelize([raw_input.first()]))
inputNoheader.count()
data = inputNoheader.map(loadRecord)

data.collect()

In [ ]:
data.filter(lambda x: x).map(lambda x: json.dumps(x)).saveAsTextFile('./Pr5.json.Gzip', compressionCodecClass = "org.apache.hadoop.io.compress.GzipCodec")
#data.filter(lambda x: x).map(lambda x: json.dumps(x)).saveAsTextFile('./Pr5.json.Lzo', compressionCodecClass = "com.hadoop.compression.lzo.LzoCodec")

#%cat ./Pr5.json.Gzip/
%ls ./Pr5.json.Gzip/
#%ls ./

In [ ]:
#data_json = sc.textFile('./Pr5.json.Gzip/part-00000.gz').map(lambda x: json.loads(x))
data_json = sc.textFile('./Pr5.json/')

data_json.collect()

In [ ]:
def writeRecords(records):
    """Write out CSV lines"""
    output = StringIO.StringIO()
    writer = csv.DictWriter(output, fieldnames=fields)#["name", "favoriteAnimal", "c", "d"]
    for record in records:
        writer.writerow(record)
    return [output.getvalue()]
data.mapPartitions(writeRecords).saveAsTextFile('Prt5csv.csv')

%cat ./Prt5csv.csv/part-0000*

In [ ]:
data_seq = sc.parallelize([("Panda", 3), ("Kay", 6), ("Snail", 2)])
data_seq.saveAsSequenceFile('./data_seq')

%cat ./data_seq/*

In [ ]:
data_seq_rel= sc.sequenceFile("./data_seq/", "org.apache.hadoop.io.Text", "org.apache.hadoop.io.IntWritable")
data_seq_rel

data_seq_rel.collect()

In [ ]:

#Prt5.txt
Name	Country	Email	OS
John	USA	john@gmail.com	Windows
Smith	Canada	smith@yahoo.com	Linux
Yan	China	yan@alibaba.com	Windows